In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

In [2]:
#Read file accounts and allocate list_accounts to be calculated
accounts = pd.read_excel("chart_of_accounts.xlsx")
list_accounts = accounts['account']

#Read file ledger *ATTENTION : only to be used if file is small, otherwise it wil take great computational time
ledger = pd.read_excel("general_ledger.xlsx")

In [3]:
#filter ledger accounts by account in list_accounts, this way it drop accounts that will not be evaluated and reduce
# computational time
ledger_filtered = ledger.query('account in @list_accounts')

#group accounts by account having the sum parameter of values
ledger_group = ledger_filtered.groupby(['account'], as_index = False).agg({'value' : 'sum'})

In [4]:
#this function determines the sum of values stored in the child and parent leaves
#df_parent : Dataframe with leaves parent and value
#df_child : Dataframe with leaves child and value
#return the Dataframe with parent leaves and the sum value of its children

#in summary, if the substring account child contain the parent substring account, then sum its values

def sum_leaves(df_parent, df_child):
    #create dict leaves to avoid duplicates, each key will allocate the account parent with its respective sum
    #values of its children
    leaves = {} 
    
    #for each parent, run its children having the last account substring matching its parent and sum the respective values
    for index_parent, leave_parent in enumerate(df_parent.iloc[:, 0] ): 
        sum_leave_parent = 0
        for index_child, leave_child in enumerate(df_child.iloc[:, 0]):
            if str(leave_parent) in str(leave_child):
                sum_leave_parent = sum_leave_parent + df_child.iloc[index_child]['value']
            leaves[leave_parent] = sum_leave_parent
            
    #create Dataframe for parents and values of its children
    df_leaves = pd.DataFrame(list(leaves.items()),columns = ['account','value']) 
    return df_leaves

In [5]:
#determinate the leaves accounts substring to be evaluated by its parents
#in summary, split the accounts strings to allocate the parents and children leaves

def account_leaves(df):
    #split each segment of account to determine the parent and child leaves
    accounts_leaves = df['account'].str.split('.',expand=True)

    #determinate the leaves (in this case, leaves3, leaves2, leaves1 and leaves0)
    accounts_leaves['leaves4'] = accounts_leaves[0] + "." + accounts_leaves[1] + "." + accounts_leaves[2] + "." + accounts_leaves[3] + "." + accounts_leaves[4]
    accounts_leaves['leaves3'] = accounts_leaves[0] + "." + accounts_leaves[1] + "." + accounts_leaves[2] + "." + accounts_leaves[3]
    accounts_leaves['leaves2'] = accounts_leaves[0] + "." + accounts_leaves[1] + "." + accounts_leaves[2]
    accounts_leaves['leaves1'] = accounts_leaves[0] + "." + accounts_leaves[1]
    accounts_leaves['leaves0'] = accounts_leaves[0]
    return accounts_leaves

In [6]:
#Determine the parent dataframe with its children values
accounts_leaves = account_leaves(ledger_group)

#create a dataframe of leaves and the values stored
df_leaves = pd.DataFrame (accounts_leaves, columns = ['leaves0', 'leaves1', 'leaves2', 'leaves3', 'leaves4'])
df_leaves['value'] = ledger_group['value']

#determine the parent values of leaves (in this case, leaves3, leaves2, leaves1 and leaves0) stored in each Dataframe
#for each leaf parent the function is run again for optimization

df_parent3 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves3', 'value']),
                       pd.DataFrame (df_leaves, columns = ['leaves4', 'value']))
df_parent2 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves2', 'value']),
                       pd.DataFrame (df_parent3, columns = ['account', 'value']))
df_parent1 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves1', 'value']),
                       pd.DataFrame (df_parent2, columns = ['account', 'value']))
df_parent0 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves0', 'value']),
                       pd.DataFrame (df_parent1, columns = ['account', 'value']))

#concatenate the parent values
df_leaves_parents = concat([df_parent3, df_parent2, df_parent1, df_parent0], ignore_index=True)

#drop nan in case there is an empty account
df_leaves_parents = df_leaves_parents[df_leaves_parents['account'].notna()]

In [7]:
#concat the leaves and the group dataframe
ledger = concat([ledger_group, df_leaves_parents])

#order the account number by ascending order
ledger = ledger.sort_values(by = 'account', ascending=True)

#create file for the ledger and sum of values
ledger.to_excel("ledger_sum_values.xlsx")

-----------------------------------------------------------------------------------------------------------------------------

This part treats big files "ledger" and treat them accordingly. In this case the format is presumed to be "ledger.csv"

In [8]:
#ONLY FOR BIG FILES

#read acocunts and allocate list accounts
accounts = pd.read_excel("chart_of_accounts.xlsx")
list_accounts = accounts['account']

#this code runs big files ""general_ledger.csv" in chunks and treat them accordingly
#in my experience treating files over 20Gb, a chunk size of 1000000 does not causes error or great computational time

iteration = 1
flag = False
for chunk in pd.read_csv("general_ledger.csv", sep = ';', chunksize=1000): #chunksize of 1000 for visualisation
    #visualise iteration
    print(iteration)
    
    #filter dataframe by account to be evaluated
    ledger_filtered = chunk.query('account in @list_accounts')
    
    #convert values to float
    ledger_filtered['value'] = ledger_filtered['value'].str.replace(',','.')
    ledger_filtered['value'] = pd.to_numeric(ledger_filtered['value'], downcast="float")
   
    #group accounts by account having the sum parameter of values
    ledger_group = ledger_filtered.groupby(['account'], as_index = False).agg({'value' : 'sum'})
    
    #if first iteration, create an auxiliar dataframe for concatenation of subsequent dataframes
    if flag:
        ledger_group = pd.concat([ledger_group,aux_df])
        ledger_group = ledger_group.groupby(['account'], as_index = False).agg({'value' : 'sum'})
        aux_df = ledger_group
    else:
        aux_df = ledger_group
        flag = True
        
    iteration = iteration + 1

1
2
3
4
5
6
7
8
9


Evaluate parent values for respective childern

In [9]:
#RUN AGAIN FOR LEAVES PARENTS
accounts_leaves = account_leaves(ledger_group)

#create a dataframe of leaves and the values stored
df_leaves = pd.DataFrame (accounts_leaves, columns = ['leaves0', 'leaves1', 'leaves2', 'leaves3', 'leaves4'])
df_leaves['value'] = ledger_group['value']

#determine the parent values of leaves (in this case, leaves3, leaves2, leaves1 and leaves0) stored in each Dataframe
#for each leaf parent the function is run again for optimization

df_parent3 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves3', 'value']),
                       pd.DataFrame (df_leaves, columns = ['leaves4', 'value']))
df_parent2 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves2', 'value']),
                       pd.DataFrame (df_parent3, columns = ['account', 'value']))
df_parent1 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves1', 'value']),
                       pd.DataFrame (df_parent2, columns = ['account', 'value']))
df_parent0 = sum_leaves(pd.DataFrame(df_leaves, columns = ['leaves0', 'value']),
                       pd.DataFrame (df_parent1, columns = ['account', 'value']))

#concatenate the parent values
df_leaves_parents = concat([df_parent3, df_parent2, df_parent1, df_parent0], ignore_index=True)
#in case there is an empty account
df_leaves_parents = df_leaves_parents[df_leaves_parents['account'].notna()]

finally concat the parents and ledger_group dataframe

In [10]:
#concat the leaves and the group dataframe
ledger = concat([ledger_group, df_leaves_parents])

#order the account number by ascending order
ledger = ledger.sort_values(by = 'account', ascending=True)

#create file for the ledger and sum of values
ledger.to_excel("ledger_sum_values.xlsx")